In [14]:
## -- This notebook has been built to compute the mean duration of the phases (Stance & Swing)
## -- in order to have a quantitative evaluation of "tricky" subjects

import csv
import numpy as np
from numpy import genfromtxt as csv_load
import pandas as pd

## -- Models and Signals parameters
model = 'FF4'
thr = 350
folds = 10
subject = 1
n_subjects = 26

Original_Subjects = False
Predicted_Subjects = True

real_subj = [n for n in range(1,n_subjects + 1)]
pred_subj = [
[20, 25, 21, 6, 4], # 1
[4, 16, 6, 15, 5], # 2
[17, 20, 10, 2, 26], # 3
[26, 9, 17, 14, 6], # 4
[7, 6, 17, 10, 3], # 5
[2, 26, 17, 24, 13], # 6
[19, 3, 7, 15, 24], # 7
[25, 4, 6, 2, 7], # 8
[23, 8, 13, 21, 14], # 9
[9, 1, 21, 24, 3] # 10
]

if Original_Subjects:
    subjects = real_subj
    in_path = '../subjects/min-max/clean/baso_real/'
    #file = in_path + 's' + str(subject) + '_baso_real.csv'

    
if Predicted_Subjects:
    subjects = pred_subj
    in_path = '../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_' + model + '/Fold_' + str(fold) + '/' 
    #file = in_path + 's' + str(subject) + '_predicted_clean_' +  str(thr) + '.csv'


def checkHSs(v):
    hss = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 1 and v[s+1] == 0):
            hss.append(count)
    return hss

def checkTOs(v):
    tos = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 0 and v[s+1] == 1):
            tos.append(count)
    return tos

in_path_log = '../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_FF4/Fold_Data/'
subj_type = '_Predicted'

with open(in_path_log + 'Stance_Swing_Durations' + subj_type + '.csv', mode='a') as data_log:
    
    for fold in range(0, folds):
        if Original_Subjects:
            header = ['Subject', 'Mean Stance', 'Stance SD', 'Mean Swing', 'Swing SD', 'Stance %', 'Swing %']
            data_log_global = csv.writer(data_log, delimiter=';')
            data_log_global.writerow(header)
        
        if Predicted_Subjects:
            header = ['Subject', 'Mean Stance', 'Stance SD', 'Mean Swing', 'Swing SD', 'Stance %', 'Swing %']
            data_log_global = csv.writer(data_log, delimiter=';')
            data_log_global.writerow(['FOLD ' + str(fold + 1)])
            data_log_global.writerow(header)
            
        for subj in subjects[fold]:
            if Original_Subjects:
                file = in_path + 's' + str(subj) + '_baso_real.csv'

            if Predicted_Subjects:
                in_path = '../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_' + model + '/Fold_' + str(fold + 1) + '/' 
                file = in_path + 's' + str(subj) + '_predicted_clean_' +  str(thr) + '.csv'



            v = csv_load(file, delimiter = ',')
            phase = 1
            phase_length = []
            count = 0

            hss = checkHSs(v)
            tos = checkTOs(v)

            hss = np.asarray(hss, dtype = int)
            tos = np.asarray(tos, dtype = int)
            
            #print(hss,tos)
            limit = min(len(hss),len(tos))
            stances = np.zeros(limit, dtype = int)
            swings = np.zeros(limit, dtype = int)


            ## -- Stance = TO[i] - HS[i]
            for i in range(0, limit):
                stances[i] = tos[i] - hss[i]

            mean_stance = np.mean(stances)
            mean_stance = np.around(mean_stance, decimals = 3)
            standard_dev_stance = np.std(stances)
            standard_dev_stance = np.around(standard_dev_stance, decimals = 3)

            print('## -- SUBJECT ' + str(subj) + ' -- ##\n')
            print('Mean Stance +/- SD: ')
            print(str(mean_stance) + ' +/- ' +  str(standard_dev_stance))

            ## -- Swing = HS[i+1] - TO[i]
            for j in range(0, limit - 1):
                swings[j] = abs(hss[j + 1] - tos[j])

            mean_swing = np.mean(swings)
            mean_swing = np.around(mean_swing, decimals = 3)
            standard_dev_swing = np.std(swings)
            standard_dev_swing = np.around(standard_dev_swing, decimals = 3)

            print('Mean Swing +/- SD: ')
            print(str(mean_swing) + ' +/- ' +  str(standard_dev_swing))

            tot = mean_swing + mean_stance
            swing_ratio = np.around((mean_swing/tot) * 100, decimals = 3)
            stance_ratio = np.around((mean_stance/tot) * 100, decimals = 3) 

            print('\nStance: ' + str(stance_ratio) + ' %' + '\nSwing: ' + str(swing_ratio) + ' %\n\n')
            
            data_log_global.writerow(['Subject ' + str(subj) , mean_stance, standard_dev_stance, mean_swing, standard_dev_swing, stance_ratio, swing_ratio])
        data_log_global.writerow([' '])

## -- SUBJECT 20 -- ##

Mean Stance +/- SD: 
1371.754 +/- 184.932
Mean Swing +/- SD: 
1051.955 +/- 170.527

Stance: 56.597 %
Swing: 43.403 %


## -- SUBJECT 25 -- ##

Mean Stance +/- SD: 
1176.64 +/- 147.531
Mean Swing +/- SD: 
981.664 +/- 142.388

Stance: 54.517 %
Swing: 45.483 %


## -- SUBJECT 21 -- ##

Mean Stance +/- SD: 
1305.513 +/- 324.255
Mean Swing +/- SD: 
1113.97 +/- 170.948

Stance: 53.958 %
Swing: 46.042 %


## -- SUBJECT 6 -- ##

Mean Stance +/- SD: 
1186.131 +/- 78.062
Mean Swing +/- SD: 
1054.187 +/- 95.087

Stance: 52.945 %
Swing: 47.055 %


## -- SUBJECT 4 -- ##

Mean Stance +/- SD: 
1194.757 +/- 149.176
Mean Swing +/- SD: 
954.559 +/- 135.747

Stance: 55.588 %
Swing: 44.412 %


## -- SUBJECT 4 -- ##

Mean Stance +/- SD: 
1313.61 +/- 333.736
Mean Swing +/- SD: 
875.509 +/- 150.573

Stance: 60.006 %
Swing: 39.994 %


## -- SUBJECT 16 -- ##

Mean Stance +/- SD: 
1343.588 +/- 91.299
Mean Swing +/- SD: 
993.951 +/- 103.838

Stance: 57.479 %
Swing: 42.521 %


## -- SUBJEC

In [6]:
subjects


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26]